## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Dec. 30th Jan. 6th


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,December-30-2024,Condo/Co-op,3404 E Gardens East Dr E Unit 25a,Palm Beach Gardens,FL,33410.0,225000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach-Gardens/3...,Beaches MLS,RX-11030658,N,Y,26.826592,-80.087428


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
df2 = df[(df['SOLD DATE'] > '2024-12-29') & (df['SOLD DATE'] < '2025-01-07')]

In [10]:
df2.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
132,PAST SALE,2025-01-06,Condo/Co-op,600 SE 5th Ave Unit 108s,Boca Raton,FL,33432.0,4400000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/600-SE-5t...,Beaches MLS,RX-10971987,N,Y,26.344627,-80.077096
30,PAST SALE,2025-01-06,Condo/Co-op,801 Lake Shore Dr #704,Lake Park,FL,33403.0,380000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/801-...,Beaches MLS,RX-11033652,N,Y,26.799882,-80.052684
180,PAST SALE,2025-01-06,Condo/Co-op,5550 Witney Dr #103,Delray Beach,FL,33484.0,239900.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/5550-Wi...,Beaches MLS,RX-11016967,N,Y,26.452345,-80.131681
211,PAST SALE,2025-01-06,Condo/Co-op,5376 Lakefront Blvd Unit B,Delray Beach,FL,33484.0,125000.0,1.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/5376-La...,Beaches MLS,RX-11003202,N,Y,26.464533,-80.127911
176,PAST SALE,2025-01-06,Condo/Co-op,111 Shore Ct Unit 209c,North Palm Beach,FL,33408.0,165000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Palm-Beach/111...,Beaches MLS,RX-11019557,N,Y,26.807731,-80.052758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,PAST SALE,2024-12-30,Condo/Co-op,227 Sussex L #2270,West Palm Beach,FL,33417.0,140000.0,2.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/227-...,Beaches MLS,RX-10948191,N,Y,26.720161,-80.134195
13,PAST SALE,2024-12-30,Condo/Co-op,17260 Boca Club 1507 Blvd #1507,Boca Raton,FL,33487.0,495000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/17260-Boc...,Beaches MLS,RX-11035598,N,Y,26.424145,-80.105823
19,PAST SALE,2024-12-30,Condo/Co-op,5500 Old Ocean Blvd #104,Ocean Ridge,FL,33435.0,550000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Ocean-Ridge/5500-Old...,Beaches MLS,RX-11027407,N,Y,26.512526,-80.050730
32,PAST SALE,2024-12-30,Condo/Co-op,2871 N Ocean Blvd Unit M540,Boca Raton,FL,33431.0,400000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2871-N-Oc...,Beaches MLS,RX-11029799,N,Y,26.376680,-80.068894


In [11]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [12]:
df_top_ten = df2.head(10)

## Collect Agent Information

In [13]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [14]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [15]:
response_df = pd.DataFrame(response_list)

In [16]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [17]:
df_top_ten = merged_df

## Current Week's Values

In [18]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

95


In [19]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$641,448


In [20]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$390


In [21]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$60,937,525


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# Define the date range
start_date = '2024-12-23'
end_date = '2024-12-29'

# Filter the DataFrame between two dates (inclusive)
old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [23]:
# comparison week total sales
print(f'Comparison week filtered sales: {(len(old_week))}')
print('-----------')
print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
print('-----------')
print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
print('-----------')
print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
print('-----------')

Comparison week filtered sales: 66
-----------
Comparison week average sales price: $507,248.14
-----------
Comparison week average psf: $331.26
-----------
Comparison week sales total: $33,478,377.00
-----------


In [24]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Palm Beach
Input Previous Week Condo Sales Total (number units sold):
66
Input Previous Week Condo Average Sales Price:
507248
Input Previous Week Condo Average PSF:
331
Input Previous Week Condo Sales total (ex: 198_000_000)
33478377


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [25]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [26]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [27]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [28]:
pd.set_option('display.max_columns',None)

In [29]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-01-03,Condo/Co-op,400 S Ocean Blvd Unit Phb,Palm Beach,FL,33480.0,5877600.0,2.0,2.5,Four Hundred SO Ocean Blvd Condo,2003.0,NaN,1965.0,NaN,2934.0,4043.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/400-S-Oce...,Beaches MLS,RX-10950672,N,Y,26.701897,-80.033669,https://www.redfin.com/FL/Palm-Beach/400-S-Oce...,Christian Angle,Christian Angle Real Estate,None,None,Crista Ryan,"Tina Fanjul Associates, Inc.",None,None,1,orange


In [30]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [31]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-01-03,Condo/Co-op,400 S Ocean Blvd Unit Phb,Palm Beach,FL,33480.0,5877600.0,2.0,2.5,Four Hundred SO Ocean Blvd Condo,2003.0,NaN,1965.0,NaN,2934.0,4043.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/400-S-Oce...,Beaches MLS,RX-10950672,N,Y,26.701897,-80.033669,https://www.redfin.com/FL/Palm-Beach/400-S-Oce...,Christian Angle,Christian Angle Real Estate,None,None,Crista Ryan,"Tina Fanjul Associates, Inc.",None,None,1,orange
1,PAST SALE,2025-01-03,Condo/Co-op,2500 S Ocean Blvd #501,Boca Raton,FL,33432.0,5650000.0,4.0,5.5,Luxuria,4424.0,NaN,2009.0,NaN,1277.0,8978.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2500-S-Oc...,Beaches MLS,RX-10936469,N,Y,26.326626,-80.074533,https://www.redfin.com/FL/Boca-Raton/2500-S-Oc...,Scot Karp,Premier Estate Properties Inc,Michael Wells,Premier Estate Properties Inc,Rochelle LeCavalier,Douglas Elliman,None,None,2,blue
2,PAST SALE,2025-01-06,Condo/Co-op,600 SE 5th Ave Unit 108s,Boca Raton,FL,33432.0,4400000.0,3.0,3.5,Mizner Grand Condo,5186.0,NaN,1999.0,NaN,848.0,6178.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/600-SE-5t...,Beaches MLS,RX-10971987,N,Y,26.344627,-80.077096,https://www.redfin.com/FL/Boca-Raton/600-SE-5t...,Jonathan Postma,Coldwell Banker/BR,None,None,Jonathan Postma,Coldwell Banker/BR,None,None,3,blue
3,PAST SALE,2024-12-30,Condo/Co-op,201 S Narcissus Ave Ph 1,West Palm Beach,FL,33401.0,4025000.0,4.0,4.5,Esplanade Grande Condo,3899.0,NaN,2004.0,NaN,1032.0,3058.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/201-...,Beaches MLS,RX-10947152,N,Y,26.711982,-80.051026,https://www.redfin.com/FL/West-Palm-Beach/201-...,Wendy Roberts,Illustrated Properties,None,None,Lisa Lipsky,Champagne & Parisi Real Estate,None,None,4,blue
4,PAST SALE,2024-12-31,Condo/Co-op,3720 S Ocean Blvd Blvd #901,Highland Beach,FL,33487.0,3100000.0,3.0,3.5,Toscana West Condo,2736.0,NaN,2001.0,NaN,1133.0,3168.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3720-...,Beaches MLS,RX-11032029,N,Y,26.408254,-80.066146,https://www.redfin.com/FL/Highland-Beach/3720-...,Virginia Dollard,Klein Morgan Realty LLC,None,None,Jonathan Lewis,Elite Realty of South Florida,None,None,5,blue
5,PAST SALE,2025-01-03,Condo/Co-op,200 N Ocean Blvd #4,Palm Beach,FL,33480.0,2800000.0,2.0,2.5,Warden House Condo,1126.0,NaN,1922.0,NaN,2487.0,3533.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/200-N-Oce...,Beaches MLS,RX-11025210,N,Y,26.721635,-80.035110,https://www.redfin.com/FL/Palm-Beach/200-N-Oce...,"Frank Herz, PA",Keller Williams Realty - Welli,None,None,Wahkuna Vega-Farrell,Compass Florida LLC (PB),None,None,6,blue
6,PAST SALE,2025-01-02,Condo/Co-op,120 S Ocean Blvd Unit 1-A,Delray Beach,FL,33483.0,2500000.0,2.0,2.0,Ocean Place Condo,1886.0,NaN,1981.0,NaN,1326.0,1541.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/120-S-O...,Beaches MLS,RX-11022225,N,Y,26.458917,-80.059459,https://www.redfin.com/FL/Delray-Beach/120-S-O...,Gary Shusas,Coldwell Banker Realty /Delray Beach,None,None,Nicholas Wisell,Coldwell Banker Realty /Delray Beach,None,None,7,blue
7,PAST SALE,2025-01-03,Condo/Co-op,155 E Boca Raton Rd Unit Th2,Boca Raton,FL,33432.0,2100000.0,3.0,3.5,Tower 155 Condo,2850.0,NaN,2020.0,NaN,737.0,3744.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/155-E-Boc...,Beaches MLS,RX-10950160,N,Y,26.351656,-80.084477,https://www.redfin.com/FL/Boca-Raton/155-E-Boc...,Ina Bloom,Compass Florida LLC,Pamela Levine,Luxury Partners Realty,Meghan Stewart,Compass Florida LLC,None,None,8,blue
8,PAST SALE,2025-01-02,Condo/Co-op,250 S Ocean Blvd Unit

In [33]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent {county} County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [34]:
m.save('index.html')

## Data snagger

In [35]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [36]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-01-03,Condo/Co-op,400 S Ocean Blvd Unit Phb,Palm Beach,FL,33480.0,5877600.0,2.0,2.5,Four Hundred SO Ocean Blvd Condo,2003.0,NaN,1965.0,NaN,2934.0,4043.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/400-S-Oce...,Beaches MLS,RX-10950672,N,Y,26.701897,-80.033669,https://www.redfin.com/FL/Palm-Beach/400-S-Oce...,Christian Angle,Christian Angle Real Estate,None,None,Crista Ryan,"Tina Fanjul Associates, Inc.",None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [37]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/PalmBeach_County_condo_sales_week_ending_01062024


# CREATE TEMPLATE 

In [38]:
muni_set = set(df_top_ten['CITY'])

In [39]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [40]:
df_top_ten.reset_index(inplace=True,drop=True)

In [41]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [42]:
top_sale

'Four Hundred SO Ocean Blvd Condo at 400 S Ocean Blvd Unit Phb in Palm Beach'

In [43]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [44]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [45]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-01-03,Condo/Co-op,400 S Ocean Blvd Unit Phb,Palm Beach,FL,33480.0,5877600.0,2.0,2.5,Four Hundred SO Ocean Blvd Condo,2003.0,NaN,1965.0,NaN,2934.0,4043.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/400-S-Oce...,Beaches MLS,RX-10950672,N,Y,26.701897,-80.033669,https://www.redfin.com/FL/Palm-Beach/400-S-Oce...,Christian Angle,Christian Angle Real Estate,None,None,Crista Ryan,"Tina Fanjul Associates, Inc.",None,None,1,orange,Four Hundred SO Ocean Blvd Condo at 400 S Ocea...
1,PAST SALE,2025-01-03,Condo/Co-op,2500 S Ocean Blvd #501,Boca Raton,FL,33432.0,5650000.0,4.0,5.5,Luxuria,4424.0,NaN,2009.0,NaN,1277.0,8978.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2500-S-Oc...,Beaches MLS,RX-10936469,N,Y,26.326626,-80.074533,https://www.redfin.com/FL/Boca-Raton/2500-S-Oc...,Scot Karp,Premier Estate Properties Inc,Michael Wells,Premier Estate Properties Inc,Rochelle LeCavalier,Douglas Elliman,None,None,2,blue,Luxuria at 2500 S Ocean Blvd #501 in Boca Raton
2,PAST SALE,2025-01-06,Condo/Co-op,600 SE 5th Ave Unit 108s,Boca Raton,FL,33432.0,4400000.0,3.0,3.5,Mizner Grand Condo,5186.0,NaN,1999.0,NaN,848.0,6178.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/600-SE-5t...,Beaches MLS,RX-10971987,N,Y,26.344627,-80.077096,https://www.redfin.com/FL/Boca-Raton/600-SE-5t...,Jonathan Postma,Coldwell Banker/BR,None,None,Jonathan Postma,Coldwell Banker/BR,None,None,3,blue,Mizner Grand Condo at 600 SE 5th Ave Unit 108s...
3,PAST SALE,2024-12-30,Condo/Co-op,201 S Narcissus Ave Ph 1,West Palm Beach,FL,33401.0,4025000.0,4.0,4.5,Esplanade Grande Condo,3899.0,NaN,2004.0,NaN,1032.0,3058.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/201-...,Beaches MLS,RX-10947152,N,Y,26.711982,-80.051026,https://www.redfin.com/FL/West-Palm-Beach/201-...,Wendy Roberts,Illustrated Properties,None,None,Lisa Lipsky,Champagne & Parisi Real Estate,None,None,4,blue,Esplanade Grande Condo at 201 S Narcissus Ave ...
4,PAST SALE,2024-12-31,Condo/Co-op,3720 S Ocean Blvd Blvd #901,Highland Beach,FL,33487.0,3100000.0,3.0,3.5,Toscana West Condo,2736.0,NaN,2001.0,NaN,1133.0,3168.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3720-...,Beaches MLS,RX-11032029,N,Y,26.408254,-80.066146,https://www.redfin.com/FL/Highland-Beach/3720-...,Virginia Dollard,Klein Morgan Realty LLC,None,None,Jonathan Lewis,Elite Realty of South Florida,None,None,5,blue,Toscana West Condo at 3720 S Ocean Blvd Blvd #...
5,PAST SALE,2025-01-03,Condo/Co-op,200 N Ocean Blvd #4,Palm Beach,FL,33480.0,2800000.0,2.0,2.5,Warden House Condo,1126.0,NaN,1922.0,NaN,2487.0,3533.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/200-N-Oce...,Beaches MLS,RX-11025210,N,Y,26.721635,-80.035110,https://www.redfin.com/FL/Palm-Beach/200-N-Oce...,"Frank Herz, PA",Keller Williams Realty - Welli,None,None,Wahkuna Vega-Farrell,Compass Florida LLC (PB),None,None,6,blue,Warden House Condo at 200 N Ocean Blvd #4 in P...
6,PAST SALE,2025-01-02,Condo/Co-op,120 S Ocean Blvd Unit 1-A,Delray Beach,FL,33483.0,2500000.0,2.0,2.0,Ocean Place Condo,1886.0,NaN,1981.0,NaN,1326.0,1541.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/120-S-O...,Beaches MLS,RX-11022225,N,Y,26.458917,-80.059459,https://www.redfin.com/FL/Delray-Beach/120-S-O...,Gary Shusas,Coldwell Banker Realty /Delray Beach,None,None,Nicholas Wisell,Coldwell Banker Realty /Delray Beach,None,None,7,blue,Ocean Place Condo at 120 S Ocean Blvd Unit 1-A...
7,PAST SALE,2025-01-03,Condo/Co-op,155 E Boca Raton Rd Unit Th2,Boca Raton,FL,33432.0,2100000.0,3.0,3.5,Tower 155 Condo,2850.0,NaN,202

In [46]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Four Hundred SO Ocean Blvd Condo closing tops Palm Beach weekly condo sales 
DEK: Top 10 sales ranged from $1,500,000 to $5,877,600
FEATURED HED:
SEO HED: Palm Beach County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach, weekly condo sales, Boca Raton, West Palm Beach, Palm Beach, Delray Beach, Highland Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County’s NEWS PEG HERE.

Brokers closed 95 condo sales totaling $60,937,525 million from Dec. 30th to Jan. 6th. The previous week, brokers closed 66 condo sales totaling $33,478,377.

Last week’s units sold for an average of $641,448, higher than the $507,248 average price fr

In [47]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [48]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [49]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [50]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-01-03,Condo/Co-op,400 S Ocean Blvd Unit Phb,Palm Beach,FL,33480.0,5877600.0,2.0,2.5,Four Hundred SO Ocean Blvd Condo,2003.0,NaN,1965.0,NaN,2934.0,4043.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/400-S-Oce...,Beaches MLS,RX-10950672,N,Y,26.701897,-80.033669,https://www.redfin.com/FL/Palm-Beach/400-S-Oce...,Christian Angle,Christian Angle Real Estate,None,None,Crista Ryan,"Tina Fanjul Associates, Inc.",None,None,1,orange,Four Hundred SO Ocean Blvd Condo at 400 S Ocea...
1,PAST SALE,2025-01-03,Condo/Co-op,2500 S Ocean Blvd #501,Boca Raton,FL,33432.0,5650000.0,4.0,5.5,Luxuria,4424.0,NaN,2009.0,NaN,1277.0,8978.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2500-S-Oc...,Beaches MLS,RX-10936469,N,Y,26.326626,-80.074533,https://www.redfin.com/FL/Boca-Raton/2500-S-Oc...,Scot Karp,Premier Estate Properties Inc,Michael Wells,Premier Estate Properties Inc,Rochelle LeCavalier,Douglas Elliman,None,None,2,blue,Luxuria at 2500 S Ocean Blvd #501 in Boca Raton
2,PAST SALE,2025-01-06,Condo/Co-op,600 SE 5th Ave Unit 108s,Boca Raton,FL,33432.0,4400000.0,3.0,3.5,Mizner Grand Condo,5186.0,NaN,1999.0,NaN,848.0,6178.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/600-SE-5t...,Beaches MLS,RX-10971987,N,Y,26.344627,-80.077096,https://www.redfin.com/FL/Boca-Raton/600-SE-5t...,Jonathan Postma,Coldwell Banker/BR,None,None,Jonathan Postma,Coldwell Banker/BR,None,None,3,blue,Mizner Grand Condo at 600 SE 5th Ave Unit 108s...
3,PAST SALE,2024-12-30,Condo/Co-op,201 S Narcissus Ave Ph 1,West Palm Beach,FL,33401.0,4025000.0,4.0,4.5,Esplanade Grande Condo,3899.0,NaN,2004.0,NaN,1032.0,3058.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/201-...,Beaches MLS,RX-10947152,N,Y,26.711982,-80.051026,https://www.redfin.com/FL/West-Palm-Beach/201-...,Wendy Roberts,Illustrated Properties,None,None,Lisa Lipsky,Champagne & Parisi Real Estate,None,None,4,blue,Esplanade Grande Condo at 201 S Narcissus Ave ...
4,PAST SALE,2024-12-31,Condo/Co-op,3720 S Ocean Blvd Blvd #901,Highland Beach,FL,33487.0,3100000.0,3.0,3.5,Toscana West Condo,2736.0,NaN,2001.0,NaN,1133.0,3168.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3720-...,Beaches MLS,RX-11032029,N,Y,26.408254,-80.066146,https://www.redfin.com/FL/Highland-Beach/3720-...,Virginia Dollard,Klein Morgan Realty LLC,None,None,Jonathan Lewis,Elite Realty of South Florida,None,None,5,blue,Toscana West Condo at 3720 S Ocean Blvd Blvd #...
5,PAST SALE,2025-01-03,Condo/Co-op,200 N Ocean Blvd #4,Palm Beach,FL,33480.0,2800000.0,2.0,2.5,Warden House Condo,1126.0,NaN,1922.0,NaN,2487.0,3533.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/200-N-Oce...,Beaches MLS,RX-11025210,N,Y,26.721635,-80.035110,https://www.redfin.com/FL/Palm-Beach/200-N-Oce...,"Frank Herz, PA",Keller Williams Realty - Welli,None,None,Wahkuna Vega-Farrell,Compass Florida LLC (PB),None,None,6,blue,Warden House Condo at 200 N Ocean Blvd #4 in P...
6,PAST SALE,2025-01-02,Condo/Co-op,120 S Ocean Blvd Unit 1-A,Delray Beach,FL,33483.0,2500000.0,2.0,2.0,Ocean Place Condo,1886.0,NaN,1981.0,NaN,1326.0,1541.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/120-S-O...,Beaches MLS,RX-11022225,N,Y,26.458917,-80.059459,https://www.redfin.com/FL/Delray-Beach/120-S-O...,Gary Shusas,Coldwell Banker Realty /Delray Beach,None,None,Nicholas Wisell,Coldwell Banker Realty /Delray Beach,None,None,7,blue,Ocean Place Condo at 120 S Ocean Blvd Unit 1-A...
7,PAST SALE,2025-01-03,Condo/Co-op,155 E Boca Raton Rd Unit Th2,Boca Raton,FL,33432.0,2100000.0,3.0,3.5,Tower 155 Condo,2850.0,NaN,202

In [51]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: Four Hundred SO Ocean Blvd Condo closing tops Palm Beach weekly condo sales 
DEK: Top 10 sales ranged from $1,500,000 to $5,877,600
FEATURED HED:
SEO HED: Palm Beach County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach, weekly condo sales, Boca Raton, West Palm Beach, Palm Beach, Delray Beach, Highland Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County’s NEWS PEG HERE.

Brokers closed 95 condo sales totaling $60,937,525 million from Dec. 30th to Jan. 6th. The previous week, brokers closed 66 condo sales totaling $33,478,377.

Last week’s units sold for an average of $641,448, higher than the $507,248 average price fr

In [59]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Palm-Beach/400-S-Ocean-Blvd-33480/unit-Phb/home/189453495


In [60]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Boca-Raton/2500-S-Ocean-Blvd-33432/unit-501/home/42666484


In [61]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Palm-Beach/227-Brazilian-Ave-33480/unit-3f/home/190072391


In [62]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Boca-Raton/250-S-Ocean-Blvd-33432/unit-9A/home/42321188


In [63]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Palm-Beach/400-S-Ocean-Blvd-33480/unit-Phb/home/189453495


In [64]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Boca-Raton/155-E-Boca-Raton-Rd-33432/unit-TH2/home/174200690


In [65]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-01-03,Condo/Co-op,400 S Ocean Blvd Unit Phb,Palm Beach,FL,33480.0,5877600.0,2.0,2.5,Four Hundred SO Ocean Blvd Condo,2003.0,NaN,1965.0,NaN,2934.0,4043.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/400-S-Oce...,Beaches MLS,RX-10950672,N,Y,26.701897,-80.033669,https://www.redfin.com/FL/Palm-Beach/400-S-Oce...,Christian Angle,Christian Angle Real Estate,None,None,Crista Ryan,"Tina Fanjul Associates, Inc.",None,None,1,orange,Four Hundred SO Ocean Blvd Condo at 400 S Ocea...
1,PAST SALE,2025-01-03,Condo/Co-op,2500 S Ocean Blvd #501,Boca Raton,FL,33432.0,5650000.0,4.0,5.5,Luxuria,4424.0,NaN,2009.0,NaN,1277.0,8978.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2500-S-Oc...,Beaches MLS,RX-10936469,N,Y,26.326626,-80.074533,https://www.redfin.com/FL/Boca-Raton/2500-S-Oc...,Scot Karp,Premier Estate Properties Inc,Michael Wells,Premier Estate Properties Inc,Rochelle LeCavalier,Douglas Elliman,None,None,2,blue,Luxuria at 2500 S Ocean Blvd #501 in Boca Raton
2,PAST SALE,2025-01-06,Condo/Co-op,600 SE 5th Ave Unit 108s,Boca Raton,FL,33432.0,4400000.0,3.0,3.5,Mizner Grand Condo,5186.0,NaN,1999.0,NaN,848.0,6178.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/600-SE-5t...,Beaches MLS,RX-10971987,N,Y,26.344627,-80.077096,https://www.redfin.com/FL/Boca-Raton/600-SE-5t...,Jonathan Postma,Coldwell Banker/BR,None,None,Jonathan Postma,Coldwell Banker/BR,None,None,3,blue,Mizner Grand Condo at 600 SE 5th Ave Unit 108s...
3,PAST SALE,2024-12-30,Condo/Co-op,201 S Narcissus Ave Ph 1,West Palm Beach,FL,33401.0,4025000.0,4.0,4.5,Esplanade Grande Condo,3899.0,NaN,2004.0,NaN,1032.0,3058.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/201-...,Beaches MLS,RX-10947152,N,Y,26.711982,-80.051026,https://www.redfin.com/FL/West-Palm-Beach/201-...,Wendy Roberts,Illustrated Properties,None,None,Lisa Lipsky,Champagne & Parisi Real Estate,None,None,4,blue,Esplanade Grande Condo at 201 S Narcissus Ave ...
4,PAST SALE,2024-12-31,Condo/Co-op,3720 S Ocean Blvd Blvd #901,Highland Beach,FL,33487.0,3100000.0,3.0,3.5,Toscana West Condo,2736.0,NaN,2001.0,NaN,1133.0,3168.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3720-...,Beaches MLS,RX-11032029,N,Y,26.408254,-80.066146,https://www.redfin.com/FL/Highland-Beach/3720-...,Virginia Dollard,Klein Morgan Realty LLC,None,None,Jonathan Lewis,Elite Realty of South Florida,None,None,5,blue,Toscana West Condo at 3720 S Ocean Blvd Blvd #...
5,PAST SALE,2025-01-03,Condo/Co-op,200 N Ocean Blvd #4,Palm Beach,FL,33480.0,2800000.0,2.0,2.5,Warden House Condo,1126.0,NaN,1922.0,NaN,2487.0,3533.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/200-N-Oce...,Beaches MLS,RX-11025210,N,Y,26.721635,-80.035110,https://www.redfin.com/FL/Palm-Beach/200-N-Oce...,"Frank Herz, PA",Keller Williams Realty - Welli,None,None,Wahkuna Vega-Farrell,Compass Florida LLC (PB),None,None,6,blue,Warden House Condo at 200 N Ocean Blvd #4 in P...
6,PAST SALE,2025-01-02,Condo/Co-op,120 S Ocean Blvd Unit 1-A,Delray Beach,FL,33483.0,2500000.0,2.0,2.0,Ocean Place Condo,1886.0,NaN,1981.0,NaN,1326.0,1541.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/120-S-O...,Beaches MLS,RX-11022225,N,Y,26.458917,-80.059459,https://www.redfin.com/FL/Delray-Beach/120-S-O...,Gary Shusas,Coldwell Banker Realty /Delray Beach,None,None,Nicholas Wisell,Coldwell Banker Realty /Delray Beach,None,None,7,blue,Ocean Place Condo at 120 S Ocean Blvd Unit 1-A...
7,PAST SALE,2025-01-03,Condo/Co-op,155 E Boca Raton Rd Unit Th2,Boca Raton,FL,33432.0,2100000.0,3.0,3.5,Tower 155 Condo,2850.0,NaN,202

## Time on Market Calculator

In [68]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2023, 11, 15) ## List (Earlier) date
date2 = datetime(2025, 1, 3) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

415


## Clean CSV for Datawrapper Chart

In [69]:
chart_df = df_top_ten

In [70]:
chart_df = chart_df.fillna(" ")

In [71]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [72]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [73]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [74]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [75]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x))
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [76]:
csv_date_string = today.strftime("%m_%d_%Y")

In [77]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")

In [79]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2025-01-03,400 S Ocean Blvd Unit Phb in Palm Beach,"$5,877,600",2,2.5,"2,003",1965,"$2,934",Beaches MLS,RX-10950672,Christian Angle Christian Angle Real Estate,"Crista Ryan Tina Fanjul Associates, Inc."
1,2025-01-03,2500 S Ocean Blvd #501 in Boca Raton,"$5,650,000",4,5.5,"4,424",2009,"$1,277",Beaches MLS,RX-10936469,Scot Karp Premier Estate Properties Inc Michae...,Rochelle LeCavalier Douglas Elliman
2,2025-01-06,600 SE 5th Ave Unit 108s in Boca Raton,"$4,400,000",3,3.5,"5,186",1999,$848,Beaches MLS,RX-10971987,Jonathan Postma Coldwell Banker/BR,Jonathan Postma Coldwell Banker/BR
3,2024-12-30,201 S Narcissus Ave Ph 1 in West Palm Beach,"$4,025,000",4,4.5,"3,899",2004,"$1,032",Beaches MLS,RX-10947152,Wendy Roberts Illustrated Properties,Lisa Lipsky Champagne & Parisi Real Estate
4,2024-12-31,3720 S Ocean Blvd Blvd #901 in Highland Beach,"$3,100,000",3,3.5,"2,736",2001,"$1,133",Beaches MLS,RX-11032029,Virginia Dollard Klein Morgan Realty LLC,Jonathan Lewis Elite Realty of South Florida
5,2025-01-03,200 N Ocean Blvd #4 in Palm Beach,"$2,800,000",2,2.5,"1,126",1922,"$2,487",Beaches MLS,RX-11025210,"Frank Herz, PA Keller Williams Realty - Welli",Wahkuna Vega-Farrell Compass Florida LLC (PB)
6,2025-01-02,120 S Ocean Blvd Unit 1-A in Delray Beach,"$2,500,000",2,2.0,"1,886",1981,"$1,326",Beaches MLS,RX-11022225,Gary Shusas Coldwell Banker Realty /Delray Bea...,Nicholas Wisell Coldwell Banker Realty /Delray...
7,2025-01-03,155 E Boca Raton Rd Unit Th2 in Boca Raton,"$2,100,000",3,3.5,"2,850",2020,$737,Beaches MLS,RX-10950160,Ina Bloom Compass Florida LLC Pamela Levine Lu...,Meghan Stewart Compass Florida LLC
8,2025-01-02,250 S Ocean Blvd Unit 9a in Boca Raton,"$1,750,000",2,2.5,"1,670",1975,"$1,048",Beaches MLS,RX-11024043,Jake Friedman Compass Florida LLC,Perry Melzer Compass Florida LLC Jorge Lorenzo...
9,2025-01-02,227 Brazilian Ave Unit 3f in Palm Beach,"$1,500,000",2,1.5,960,1970,"$1,563",Beaches MLS,RX-11020074,Brooke Murphy Compass Florida LLC (PB),Non-Member Non-Member Selling Office
